In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from typing_extensions import Annotated

In [2]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")
class ChatState(TypedDict):
    #add on messages
    messages:Annotated[list[BaseMessage],add_messages]

def start_chat(state:ChatState) -> ChatState:
    message=state['messages']
    response=llm.invoke(message)
    return {'messages': response}
check_pointer=MemorySaver()
graph=StateGraph(ChatState)

In [3]:
graph.add_node("start_chat",start_chat)
graph.add_edge(START,"start_chat")
graph.add_edge("start_chat",END)

In [4]:
chatbot=graph.compile(checkpointer=check_pointer)
# thread=chatbot.get_state("user123")
# chatbot.update_state("user123",{"messages":[])
thread_ID='user123'

In [5]:
while True:
    User_input=input("enter your query")
    print("USER:",User_input)
    if User_input.strip().lower() in ['end','quit','stop']:
        break
    config={"configurable":{"thread_id":thread_ID}}
    response=chatbot.invoke({"messages":[HumanMessage(content=User_input)]},config=config)
    print("AI:", response['messages'][-1].content)
    

USER: hi how are you
AI: I am doing well, thank you for asking! How are you today?
USER: my nae is sam what is your name
AI: It's nice to meet you, Sam! I don't have a name. I'm a large language model, and I don't have personal attributes like a name. You can just call me AI or whatever you like. :)
USER: stop
